In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
data = pd.read_excel("../raw/Mobility/England/myebtablesuk20112022.xlsx", sheet_name=4, skiprows=1, usecols=["laname23","country","sex","age","population_2021"])
data

In [ ]:
with open("../raw/Mobility/England/Local_Authority_Districts_May_2023_UK_BGC_V2_606764927733448598.geojson") as file:
    what = json.load(file)
needed = [el["properties"] for el in what["features"]]
locations = pd.DataFrame(needed).set_index("LAD23NM")
locations

In [ ]:
np.isin(data["laname23"], locations.index).all(), np.isin(locations.index, data["laname23"]).all()

In [ ]:
relevant = data.loc[data["country"]=="E"].groupby("laname23").agg({"population_2021":"sum"})

In [ ]:
for colname in ["LAD23CD","LONG","LAT","GlobalID"]:
    relevant[colname] = locations[colname]

In [ ]:
relevant

In [ ]:
def computeDistance(lat, long):
	lat = np.radians(lat.to_numpy())
	long = np.radians(long.to_numpy())
	distance = np.zeros((lat.shape[0],)*2)
	for i in range(lat.shape[0]):
		for j in range(i+1,lat.shape[0]):
			distance[i,j] = 6371 * np.arccos(np.sin(lat[i])*np.sin(lat[j])+np.cos(lat[i])*np.cos(lat[j])*np.cos(long[i]-long[j]))
			distance[j,i] = distance[i,j]
	return distance

In [ ]:
distances = computeDistance(relevant["LAT"], relevant["LONG"])

In [ ]:
relevant["population_2021"].to_csv("Ns/england.csv", index=False)

In [ ]:
pd.DataFrame(distances, columns=relevant.index).to_csv("distances/england.csv", index=False)